### Cropping an existing position list for manual re-imaging

This notebook 'crops' a position list generated by the HCS Site Generator plugin by selecting a subset of the positions in the position list according to a user-specified list of wells. It then orders these positions according to the order of the wells in the user-specified list and saves the positions in a new position list.

This is intended for manual redos in which the user selects a small number of wells to be re-imaged. Note that the order of these wells matters and is determined by the user in order to minimize stage movements, so it is important that the well order be preserved in the new position list. 

In [ ]:
import os
import re
import sys
import json
import numpy as np
import pandas as pd

from scipy import interpolate
from matplotlib import pyplot as plt

sys.path.insert(0, '..')
from dragonfly_automation import utils

%load_ext autoreload
%autoreload 2

In [ ]:
# test filepath
position_list_filepath = (
    '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/'
    'HCS_sites_20191009_INTERPOLATED.pos'
)
with open(position_list_filepath, 'r') as file:
    position_list = json.load(file)

### Set the path to the exported position list

Generally, this position list should be the interpolated position list generated by the stage-position-interpolation notebook. Its filename should be of the form `'{date}_raw_positions_interpolated.pos'` and it should have been saved in the folder `D:\MLPipeline\position-lists\`. 


In [ ]:
position_list_filename = '_raw_positions_interpolated.pos'

position_list_filepath = os.path.join('D:', 'PipelineML', 'position-lists', position_list_filename)
with open(position_list_filepath, 'r') as file:
    position_list = json.load(file)

### OPTION 1: Manually define a list of wells to be re-imaged

Here, we define the subset of wells to be re-imaged and the number of positions to visit in each well. This number must be less than or equal to the number of sites per well in the original position list. For example, if a 6x6 grid was used in the HCS Site Generator, then the number of sites per well would have to be 36 or less.

__Please note: the order of the `well_ids` list is critical, because it is used to determine the order of the positions in the new position list, which in turn determines the order in which the microscope will visit the wells.__ Please make sure that the order you choose will not require any long* stage movements (if necessary, you can 'pad' the list with intermediate wells to break up any such long movements).

*generally, 'long' means a distance of more than five wells

In [ ]:
selected_well_ids = ['B2']
num_sites_per_well = 36

In [ ]:
# run this cell without modification
selected_positions = []
all_positions = position_list['POSITIONS']
for selected_well_id in selected_well_ids:
    for position in all_positions:
        well_id, site_num = utils.parse_hcs_site_label(position['LABEL'])
        if well_id == selected_well_id and site_num < num_sites_per_well:
            selected_positions.append(position)
            
print('%s positions selected' % len(selected_positions))

In [ ]:
# check for well_ids that were not found in the position list
missing_well_ids = (
    set(selected_well_ids)
    .difference([utils.parse_hcs_site_label(p['LABEL'])[0] for p in selected_positions])
)
if missing_well_ids:
    print('Warning: no positions were found for these well_ids: %s' % list(missing_well_ids))
else:
    print('Positions were found for all selected wells')

In [ ]:
# save the cropped position list
cropped_position_list = position_list.copy()
cropped_position_list['POSITIONS'] = selected_positions

dst_filepath = position_list_filepath.replace('.pos', '_cropped.pos')
with open(dst_filepath, 'w') as file:
    json.dump(cropped_position_list, file)

In [ ]:
# final sanity check: view the position labels explicitly
[p['LABEL'] for p in selected_positions]

### OPTION 2: Use the FOV scores from an existing acquisition

Here, we crop the position list by only retaining the top-scoring positions in each well. The FOV scores are obtained from an existing acquisition, which must exist, and is assumed to be of the same plate. 

In [ ]:
# define the path to the PML directory for the existing acquisition
acquisition_dir = '/Users/keith.cheveralls/projects/dragonfly-automation/tests/output/PML0000-7'

# define the number of highest-scoring positions in each well
num_positions_per_well = 4

score_log = pd.read_csv(os.path.join(exp_dir, 'logs', 'fov-scoring', 'fov-score-log.csv'))
top_scores = (
    score_log.sort_values(by=['position_well_id', 'score'], ascending=False)
    .groupby('position_well_id')
    .head(num_positions_per_well)
)

In [ ]:
# run this cell without modification
selected_positions = [
    position for position in position_list['POSITIONS']
    if position['LABEL'] in top_scores.position_label.values
]

print('%s positions selected' % len(selected_positions))

In [ ]:
# check for positions that were not found in the position list
missing_position_labels = set(top_scores.position_label.values).difference([p['LABEL'] for p in selected_positions])
if missing_position_labels:
    print('Warning: these positions were not found in the position list: %s' % list(missing_position_labels))
else:
    print('All positions were found in the position list')

In [ ]:
# final sanity check: view the position labels explicitly
[p['LABEL'] for p in selected_positions]

In [ ]:
# save the cropped position list
cropped_position_list = position_list.copy()
cropped_position_list['POSITIONS'] = selected_positions

dst_filepath = position_list_filepath.replace('.pos', '_cropped.pos')
with open(dst_filepath, 'w') as file:
    json.dump(cropped_position_list, file)